In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Convolution1D, Dropout, MaxPooling1D, BatchNormalization, \
    Activation, Reshape, GlobalMaxPooling1D, LSTM, GlobalAveragePooling1D, AveragePooling1D, SeparableConvolution1D, \
    Concatenate, Layer, Conv2D
from tensorflow.keras.models import Sequential
import random
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD


# 加载数据集

In [ ]:
def DataGenerateOnehotShuffle(path,num_class):
    """
    生成数据集，onehot编码，打乱数据
    :param path:
    :param num_class:
    :return:
    """
    data=pd.read_csv(path)
    #data=data.iloc[1:,1:] #只读取第一列以后
    data_array=np.array(data,dtype=float)
    label=data_array[:,-1]
    feature = data_array[:, :-1]
    print("数据集数量统计：",len(feature),len(label))

    label_onehot=tf.keras.utils.to_categorical(label,num_classes=num_class)
    index=[i for i in range(len(feature))]
    np.random.shuffle(index)
    feature=feature[index]
    label_onehot=label_onehot[index]
    return feature,label_onehot

feature,label = DataGenerateOnehotShuffle("/content/mzp_data_c10_stride1000.csv",10)
feature = np.expand_dims(feature,axis=1)
print(feature[0])
print(feature.shape)

数据集数量统计： 999 999
[[-0.3804056   0.20388981  0.15815246 ...  0.04832353 -0.45243126
   0.08047464]]
(999, 1, 2048)


In [ ]:
def MyModel(num_class:int):
    # 16*64 维数是16 特征长度是64
    input = tf.keras.Input(shape=(1,2048))  # 2048
    x = Convolution1D(16, 3, 1, input_shape=(1, 2048), padding="same", activation="tanh", name="CONV1")(
        input)  # 第一层卷积层 4096*1--->256*16
    x = BatchNormalization()(x)
    x = MaxPooling1D(2, strides=2, padding="same")(x)


    # 第三层卷积开始
    x = Convolution1D(32, 3, 1, padding="same", activation="tanh", name="CONV3")(x)
    x = MaxPooling1D(2, strides=2, padding="same")(x)


    x = Convolution1D(64, 3, 1, padding="same", activation="tanh", name="CONV6")(x)
    x = MaxPooling1D(2, strides=2, padding="same")(x)  # 输出的shape= 64*8

    # 展平 连接全连接层
    x = GlobalAveragePooling1D()(x)  # 由全局平均池化代替全连接层
    output = Dense(num_class, activation="softmax")(x)
    model = tf.keras.Model(input, output)
    print(model.summary())

    return model

model = MyModel(10)
model.compile(optimizer=Adam(learning_rate=1e-3), loss="categorical_crossentropy",
                  metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, random_state=0)  # 切割数据集

model.fit(x_train, y_train, batch_size=256, epochs=300, validation_data=(x_test, y_test))
model.save("myModel.h5")

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1, 2048)]         0         
                                                                 
 CONV1 (Conv1D)              (None, 1, 16)             98320     
                                                                 
 batch_normalization_11 (Ba  (None, 1, 16)             64        
 tchNormalization)                                               
                                                                 
 max_pooling1d_30 (MaxPooli  (None, 1, 16)             0         
 ng1D)                                                           
                                                                 
 CONV3 (Conv1D)              (None, 1, 32)             1568      
                                                                 
 max_pooling1d_31 (MaxPooli  (None, 1, 32)             0  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 推理看下结果

In [ ]:
model = tf.keras.models.load_model("/content/myModel.h5")
x,y = feature[500],label[500]
print(x,y)
print(x.shape)
x = np.expand_dims(x,axis=0)
result = model.predict(x)
print(result)

[[ 0.4816097  -0.29558593  0.23526497 ...  0.1681389   0.43470074
  -0.26565669]] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(1, 2048)
1/1 [==============================] - 0s 111ms/step
[[1.3940348e-04 5.8273403e-05 1.3903981e-04 1.3068260e-06 2.7632745e-05
  4.4260396e-06 1.9263932e-06 2.3259676e-05 1.4241329e-05 9.9959046e-01]]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
flatbuffer_size = open("sequence.tflite", "wb").write(tflite_model)

print('The size of the converted flatbuffer is: %d bytes' % flatbuffer_size)

The size of the converted flatbuffer is: 435488 bytes


In [ ]:
!apt-get -qq install xxd
!xxd -i sequence.tflite > sequence.cc